In [27]:
# !pip install textblob

# !python -m textblob.download_corpora
# import nltk

# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# /root/nltk_data
# !rm -r /root/nltk_data

# if missing some required data
#!git clone https://github.com/nltk/nltk_data.git
#!unzip ./nltk_data/packages/tokenizers/punkt_tab.zip
#!unzip ./nltk_data/packages/tokenizers/punkt.zip
# !rm -r /opt/anaconda3/envs/OnlineReview/nltk_data
# !cp -r ./nltk_data/packages /opt/anaconda3/envs/OnlineReview/nltk_data

In [2]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
from textblob import TextBlob

In [4]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import models, transforms
from collections import OrderedDict
from tqdm import tqdm
import pprint
import cv2
import random
import torch
import sys
%matplotlib inline
sys.path.append("..")

/opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Calculate Aesthetic Quality

In [8]:
from model.resnet_FT import ResNetGAPFeatures as Net
from utils.data import read_data, create_dataloader, AestheticsDataset

## Device choice

In [5]:
use_cuda = torch.cuda.is_available()
print(use_cuda)
use_mps = torch.backends.mps.is_available()  # mac
print(use_mps)

False
True


In [6]:
# def device_set(tensor):
# resnet.to("mps")

## Load Model and Dataset

In [10]:
# !cp ./resnet50-0676ba61.pth /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
save_path = "./checkpoint/001" 
checkpoint = "epoch_19.loss_0.40747442577863185.pth"
resnet = models.resnet50(pretrained=True)
net = Net(resnet, n_features=12)
if use_cuda:
    resnet = resnet.cuda()
    net = net.cuda()
    net.load_state_dict(torch.load(f"{save_path}/{checkpoint}"))
elif use_mps:
    resnet = resnet.to("mps")
    net = net.to("mps")
    net.load_state_dict(torch.load(f"{save_path}/{checkpoint}", map_location='mps'))
else:
    net.load_state_dict(torch.load(f"{save_path}/{checkpoint}", map_location=lambda storage, loc: storage))

/opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/var/folders/3t/b9skt5wj5nx3pcjbmhb1sy100000gn/T/ipykernel_88264/1489681419.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See

In [11]:
import time
t0 = time.time()
test = read_data("./image_data/review_data_test.csv", "../datasets/yelp_sfs_15g/res_images", is_test=True)
test_loader = create_dataloader(test, batch_size=1, is_train=False)
print(round(time.time() - t0, 2))

1098.85


## Predict

In [13]:
# attr_keys = ['BalacingElements', 'ColorHarmony', 'Content', 'DoF',
#              'Light', 'MotionBlur', 'Object', 'RuleOfThirds', 'VividColor']
# non_neg_attr_keys = ['Repetition', 'Symmetry', 'score']
# all_keys = attr_keys + non_neg_attr_keys
# used_keys = ["ColorHarmony", "Content", "DoF", "Object", "VividColor", "score"]

In [14]:
# def extract_prediction(inp, net):
#     d = dict()
#     net.eval()
#     output = net(inp)
#     # print(output)
#     for i, key in enumerate(all_keys):
#         # print(output[:, i])
#         # print(output[:, i].squeeze())
#         # d[key] = output[:, i].squeeze().data[0]
#         d[key] = output[:, i].squeeze().item()
#     return d

In [15]:
# def sample_data(dataset, image_path=None):
#     idx = random.sample(range(len(dataset)), 1)[0]
#     return dataset[idx]

In [16]:
test_dataset = AestheticsDataset(test, is_train=False)

In [17]:
len(test_dataset)

125677

In [18]:
# data = test_dataset[0]
# image = data['image']
# image_path = data['image_path']
# image_default = mpimg.imread(image_path)
# img_shape = image_default.shape
# h, w = img_shape[0], img_shape[1]
# inp = Variable(image).unsqueeze(0)
# net.eval()
# output = net(inp)
# output

In [23]:
data

{'image': tensor([[[ 0.8789,  0.8447,  0.6734,  ..., -1.0390, -1.0904, -1.2445],
          [ 0.7077,  0.7933,  0.6563,  ..., -0.9020, -0.9534, -1.0904],
          [ 0.9132,  0.7248,  0.5878,  ..., -0.6965, -0.8849, -1.0390],
          ...,
          [ 0.2967,  0.5536,  0.6049,  ..., -0.1314, -0.2684, -0.2513],
          [ 0.4337,  0.5878,  0.6392,  ..., -0.3027, -0.3712, -0.4911],
          [ 0.4337,  0.4851,  0.5878,  ..., -0.4226, -0.4739, -0.4568]],
 
         [[-0.1625, -0.1625, -0.3025,  ..., -1.6506, -1.6681, -1.8081],
          [-0.3200, -0.1975, -0.2850,  ..., -1.5805, -1.5980, -1.7031],
          [-0.0924, -0.2675, -0.3725,  ..., -1.4055, -1.5105, -1.5980],
          ...,
          [-0.5826, -0.3725, -0.3725,  ..., -0.6001, -0.7577, -0.7577],
          [-0.4776, -0.3725, -0.3550,  ..., -0.8627, -0.9503, -1.0728],
          [-0.4426, -0.4251, -0.3725,  ..., -0.9853, -1.0553, -1.0553]],
 
         [[-1.2641, -1.1770, -1.2119,  ..., -1.6999, -1.6476, -1.7347],
          [-1.4733,

In [21]:
data = test_dataset[0]
image = data['image']
image_path = data['image_path']
image_default = mpimg.imread(image_path)
img_shape = image_default.shape
h, w = img_shape[0], img_shape[1]
# inp = Variable(image).unsqueeze(0)
inp = torch.tensor(image).unsqueeze(0)  # 直接转换为 Tensor，并增加 batch 维度
if use_cuda:
    inp = inp.cuda()
elif use_mps:
    inp = inp.to("mps")
# predicted_values = extract_prediction(inp, net)
score = net(inp)[:, -1].squeeze().item()
score

/var/folders/3t/b9skt5wj5nx3pcjbmhb1sy100000gn/T/ipykernel_88264/1140049197.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inp = torch.tensor(image).unsqueeze(0)  # 直接转换为 Tensor，并增加 batch 维度


0.5074242949485779

In [29]:
net.eval()  # 设置模型为评估模式（只需要一次）
for j in range(13):
    AQ_results_i = []
    for i in range(j * 10000, max(j * 10000 + 10000, len(test_dataset))):
        data = test_dataset[i]
        image = data['image']
        image_path = data['image_path']
        image_default = mpimg.imread(image_path)
        img_shape = image_default.shape
        h, w = img_shape[0], img_shape[1]
        # inp = Variable(image).unsqueeze(0)
        inp = torch.tensor(image).unsqueeze(0)  # 直接转换为 Tensor，并增加 batch 维度
        if use_cuda:
            inp = inp.cuda()
        elif use_mps:
            inp = inp.to("mps")
        # predicted_values = extract_prediction(inp, net)
        score = net(inp)[:, -1].squeeze().item()
        AQ_results_i.append({
            'review_id': data['review_id'],
            'image_file': data['image_file'],
            'score': score
        })
        if i % 1000 == 0:
            print(f"now data: {i}")
    print("I have to rest")
    time.sleep(10)
    pd.DataFrame(AQ_results_i).to_csv(f"./image_data/predict_{j}.csv")
    print(f"predict_{j} saved.")
    time.sleep(15)
        # aq_df = pd.concat(
        #     [aq_df, pd.DataFrame({'review_id': [data['review_id']], 'image': [data['image']], 'score': [score]})],
        #     axis=0, ignore_index=True)

/var/folders/3t/b9skt5wj5nx3pcjbmhb1sy100000gn/T/ipykernel_88264/2961356167.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inp = torch.tensor(image).unsqueeze(0)  # 直接转换为 Tensor，并增加 batch 维度


now data: 0
now data: 1000
now data: 2000
now data: 3000
now data: 4000
now data: 5000
now data: 6000
now data: 7000
now data: 8000
now data: 9000
I have to rest
predict_0 saved.
now data: 10000
now data: 11000
now data: 12000
now data: 13000
now data: 14000
now data: 15000
now data: 16000
now data: 17000
now data: 18000
now data: 19000
I have to rest
predict_1 saved.
now data: 20000
now data: 21000
now data: 22000
now data: 23000
now data: 24000
now data: 25000
now data: 26000
now data: 27000
now data: 28000
now data: 29000
I have to rest
predict_2 saved.
now data: 30000
now data: 31000
now data: 32000
now data: 33000
now data: 34000
now data: 35000
now data: 36000
now data: 37000
now data: 38000
now data: 39000
I have to rest
predict_3 saved.
now data: 40000
now data: 41000
now data: 42000
now data: 43000
now data: 44000
now data: 45000
now data: 46000
now data: 47000
now data: 48000
now data: 49000
I have to rest
predict_4 saved.
now data: 50000
now data: 51000
now data: 52000
now d

IndexError: single positional indexer is out-of-bounds

In [31]:
# for i in range(120000, len(test_dataset)):
#     data = test_dataset[i]
#     image = data['image']
#     image_path = data['image_path']
#     image_default = mpimg.imread(image_path)
#     img_shape = image_default.shape
#     h, w = img_shape[0], img_shape[1]
#     # inp = Variable(image).unsqueeze(0)
#     inp = torch.tensor(image).unsqueeze(0)  # 直接转换为 Tensor，并增加 batch 维度
#     if use_cuda:
#         inp = inp.cuda()
#     elif use_mps:
#         inp = inp.to("mps")
#     # predicted_values = extract_prediction(inp, net)
#     score = net(inp)[:, -1].squeeze().item()
#     AQ_results_i.append({
#         'review_id': data['review_id'],
#         'image_file': data['image_file'],
#         'score': score
#     })
#     if i % 1000 == 0:
#         print(f"now data: {i}")
# print("I have to rest")
# time.sleep(10)
# pd.DataFrame(AQ_results_i).to_csv(f"./image_data/predict_12.csv")
# print(f"predict_12 saved.")

/var/folders/3t/b9skt5wj5nx3pcjbmhb1sy100000gn/T/ipykernel_88264/3884993299.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inp = torch.tensor(image).unsqueeze(0)  # 直接转换为 Tensor，并增加 batch 维度


now data: 120000
now data: 121000
now data: 122000
now data: 123000
now data: 124000
now data: 125000
I have to rest
predict_12 saved.


# Read data and constitute regression data

In [50]:
df = pd.read_csv('review_data.csv')
df

,review_id,business_id,business,city,business_rating,business_review_count,user_id,user,user_review_count,user_totalPhotos,user_friend_count,review_time,rating,helpfulness_votes,funny_votes,cool_votes,photo_ids,review
0,h-DEP7jchR5ddJwIcKWfKg,QZQGRJZ4bZCmNoXjs3z1XQ,State Bird Provisions,San Francisco,4.0,2907.0,HToqdBghDzj5TdfqXuE1Rw,Noelle H.,862.0,3285.0,802.0,6/30/2014,4.0,1.0,0.0,3.0,"--0N47OO74edFVoZI7lGPQ,0Hpev-mgRhhQ6_Y4BPkdoQ,...",I had to wake up at 3:55am 2 months in advance...
1,A3NeldVTkSZhBIpHTG8nLA,x2djtavZIsv84ZKnz_vRMA,Beretta,San Francisco,4.0,218.0,jQb8dI1KDDTbQgI61q09tw,Ada H.,248.0,692.0,738.0,11/17/2020,5.0,3.0,1.0,3.0,"--1nspslpOXWrxMuOzXHWQ,iLIN8-peJ9PNjl3epdDp2A",The food here is delicious but the customer se...
2,ktdpmCGlzDkQCrCCLylrIg,f-m7-hyFzkf0HSEeQ2s-9A,Fog Harbor Fish House,San Francisco,4.5,7298.0,rXhwl09R9f6aiZtqxEImKA,Marian M.,812.0,5697.0,52.0,4/6/2019,4.0,0.0,0.0,0.0,"--25-yKa8cMeeqkb_-uMAA,kcmm47LgZLOkhOrqtdFWSg,...",Very popular place at Pier 39 in San Francisco...
3,3RmI4QP4sXvNwZzOCzec5g,RNy3_hU1N2qyS5PVc9RaYQ,Perilla,San Francisco,4.0,1422.0,WUAEbUYl5KV-dLZG-iei1A,Janine T.,2.0,7.0,1.0,5/27/2021,4.0,1.0,0.0,0.0,"--33tXoB9VJNn-VrppifGg,-qRf8sbAD5qUL-1eC5PgNw,...",We were at golden gate park and just looking f...
4,G67nazKpUS7ZYNEepeMXAA,lJAGnYzku5zSaLnQ_T6_GQ,Brenda's French Soul Food,San Francisco,4.0,11421.0,0E4kP4nuKUHhobjdXxSE_A,Stacy X.,115.0,238.0,234.0,9/10/2012,5.0,2.0,0.0,2.0,"--41iNhhL_NbtahcbQ7eHg,1U1NJkbXXxCTbK-cVyZz3g,...","I LOVE THIS RESTAURANT.<br>As with much of SF,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42934,dG6SJlgd-rBNFWSfmgxUWA,QQkroHtwS96-dzMpT4a-DA,Delarosa,San Francisco,4.0,1730.0,dF8ObA8Nxe4JBdfhDdvv2Q,Michael S.,130.0,24.0,784.0,1/21/2017,5.0,1.0,1.0,3.0,__jDVW3vOm23LHQgZuwUgQ,Stopped by to split a Margherita Burrata pie a...
42935,mOUegi6l8LoVZHDwpXB5YA,8kck3-K4zYKTJbJko0JlXQ,Farmhouse Kitchen Thai Cuisine,San Francisco,4.0,3092.0,ntEOF5qg9ulhUdkBnNei-g,Elana R.,62.0,60.0,1.0,10/22/2016,4.0,0.0,0.0,0.0,__YU2o_dMH-iSLFTAhDd0A,Very busy and had to wait 40 min even w ressie...
42936,uCkQoCOLEgD8LiidaRN_yg,UTVDbZv-qiHU06sDkoel3g,Chez Maman East,San Francisco,4.5,2211.0,1qqwHZ0Bf1f1O5CUjo3PnA,Randy A.,73.0,67.0,105.0,8/15/2017,5.0,0.0,0.0,0.0,__z2-79wN6Nz4alwyRNYLA,"My wife, 12 year old daughter, and I came to C..."
42937,cgDvrvyB69_tYyW2c3TxaQ,-DrR38H1Abk0wCyu9XOLug,Sweet Maple,San Francisco,4.5,4210.0,gr4GyG6Y29NjWyEtx2A1iQ,Shawna N.,30.0,72.0,4.0,8/4/2021,5.0,1.0,0.0,0.0,__ZtrB8IXulXhOifmmXyIQ,Such a great breakfast/brunch spot! Went on th...


## Load Aesthetic Quality

In [52]:
AQ_df = pd.DataFrame(columns=["review_id", "image_file", "score"])
for i in range(13):
    df_i = pd.read_csv(f"./image_data/predict_{i}.csv", header=0, index_col=0)
    # print(df_i)
    AQ_df = pd.concat([AQ_df, df_i], axis=0, ignore_index=True)
AQ_df

,review_id,image_file,score
0,h-DEP7jchR5ddJwIcKWfKg,--0N47OO74edFVoZI7lGPQ.jpg,0.513821
1,h-DEP7jchR5ddJwIcKWfKg,0Hpev-mgRhhQ6_Y4BPkdoQ.jpg,0.600503
2,h-DEP7jchR5ddJwIcKWfKg,3oz-yQlzresCPQ4RWtc3og.jpg,0.546512
3,h-DEP7jchR5ddJwIcKWfKg,B0TKyNStkgreP_kCQl3K7g.jpg,0.593138
4,h-DEP7jchR5ddJwIcKWfKg,bfPlnAws7z1-LjX5nDlD3Q.jpg,0.568949
...,...,...,...
131349,dG6SJlgd-rBNFWSfmgxUWA,__jDVW3vOm23LHQgZuwUgQ.jpg,0.629269
131350,mOUegi6l8LoVZHDwpXB5YA,__YU2o_dMH-iSLFTAhDd0A.jpg,0.776564
131351,uCkQoCOLEgD8LiidaRN_yg,__z2-79wN6Nz4alwyRNYLA.jpg,0.425285
131352,cgDvrvyB69_tYyW2c3TxaQ,__ZtrB8IXulXhOifmmXyIQ.jpg,0.557392


In [53]:
AQ_final = AQ_df.groupby(['review_id'])["score"].mean().reset_index()
AQ_final

,review_id,score
0,--CobcZar8Hf8CP8eHbyeQ,0.735495
1,--MT_7zMKUlQNyO9H_KFZg,0.628015
2,--TqWD0GE5D_rwoLf5N2wQ,0.603969
3,--Xz1l0UuB7lojstMlatdw,0.668892
4,--_74_HireEhXErzHY9Q9A,0.454881
...,...,...
42934,zz_aOkrWB_xnqquSdcg-Lw,0.707450
42935,zzeRn1rBE83snYehQiv7RA,0.714123
42936,zzhsUryWKU9w9sk691CAsg,0.775594
42937,zzyzEv00h2ZbA97H5mlZdQ,0.607561


In [54]:
v_df = pd.DataFrame()
v_df['review_id'] = df['review_id']
# Aesthetic Quality  sum from p (score_p) / No. of photos of review i
v_df = pd.merge(v_df, AQ_final, on='review_id', how='left')

v_df

,review_id,score
0,h-DEP7jchR5ddJwIcKWfKg,0.572857
1,A3NeldVTkSZhBIpHTG8nLA,0.544281
2,ktdpmCGlzDkQCrCCLylrIg,0.598304
3,3RmI4QP4sXvNwZzOCzec5g,0.788552
4,G67nazKpUS7ZYNEepeMXAA,0.651584
...,...,...
42934,dG6SJlgd-rBNFWSfmgxUWA,0.638181
42935,mOUegi6l8LoVZHDwpXB5YA,0.797839
42936,uCkQoCOLEgD8LiidaRN_yg,0.438558
42937,cgDvrvyB69_tYyW2c3TxaQ,0.535849


In [55]:
# 计算 subjectivity 和 polarity
v_df['subjectivity'] = df['review'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
v_df['polarity'] = df['review'].apply(lambda x: TextBlob(x).sentiment.polarity)

# 计算 AO 和 SE
v_df['AO'] = 1 - v_df['subjectivity']  # Argument Objectivity  1 - subjectivity_i  ranges from 0 to 1
v_df['SE'] = abs(v_df['polarity'] - 0.278)  # Sentiment Extremity   |sentiment_polarity_i - 0.278|  # sentiment polarity ranges from -1 to 1 where 0.278 is the average sentiment across all reviews
# Type_i = 1 if rating is 4 or 5 stars, else 0 (negative)
v_df['Type'] = np.where(df['rating'] >= 4, 1, 0)
# Perceived helpfulness. Number of “useful” votes review i received.
v_df['HV'] = df['helpfulness_votes']
# Number of photos that review i contains.
v_df['NoP'] = df['photo_ids'].str.split(',').str.len()  # v_df['NoP'] = df['photo_ids'].apply(lambda x: len(x.split(',')))
# Number of words in the text of review i
v_df['words'] = df['review'].apply(lambda x: len(TextBlob(x).words))
# Number of days elapsed since review i was posted.
v_df['days'] = (datetime.now() - pd.to_datetime(df['review_time'], format='%m/%d/%Y')).dt.days
# Number of reviewer u’s friends.
v_df['NoF'] = df['user_friend_count']
# Number of reviews the reviewer u posted.
v_df['NoR'] = df['user_review_count']
# Volume of consumer reviews of restaurant b.
v_df['volume_b'] = df['business_review_count']
# Overall consumer rating of restaurant b
v_df['b_rating'] = df['business_rating']
v_df

,review_id,score,subjectivity,polarity,AO,SE,Type,HV,NoP,words,days,NoF,NoR,volume_b,b_rating
0,h-DEP7jchR5ddJwIcKWfKg,0.572857,0.564444,0.297187,0.435556,0.019187,1,1.0,10,237,3825,802.0,862.0,2907.0,4.0
1,A3NeldVTkSZhBIpHTG8nLA,0.544281,0.680128,0.405449,0.319872,0.127449,1,3.0,2,124,1493,738.0,248.0,218.0,4.0
2,ktdpmCGlzDkQCrCCLylrIg,0.598304,0.640000,0.539333,0.360000,0.261333,1,0.0,4,96,2084,52.0,812.0,7298.0,4.5
3,3RmI4QP4sXvNwZzOCzec5g,0.788552,0.533333,0.277778,0.466667,0.000222,1,1.0,3,20,1302,1.0,2.0,1422.0,4.0
4,G67nazKpUS7ZYNEepeMXAA,0.651584,0.586975,0.270432,0.413025,0.007568,1,2.0,8,267,4483,234.0,115.0,11421.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42934,dG6SJlgd-rBNFWSfmgxUWA,0.638181,0.681667,0.285833,0.318333,0.007833,1,1.0,1,85,2889,784.0,130.0,1730.0,4.0
42935,mOUegi6l8LoVZHDwpXB5YA,0.797839,0.723333,0.680000,0.276667,0.402000,1,0.0,1,39,2980,1.0,62.0,3092.0,4.0
42936,uCkQoCOLEgD8LiidaRN_yg,0.438558,0.626364,0.353679,0.373636,0.075679,1,0.0,1,162,2683,105.0,73.0,2211.0,4.5
42937,cgDvrvyB69_tYyW2c3TxaQ,0.535849,0.545294,0.425343,0.454706,0.147343,1,1.0,1,111,1233,4.0,30.0,4210.0,4.5


## logarithm and z-score normalization

In [57]:
v_df_log = pd.DataFrame()
v_df_log["review_id"] = v_df["review_id"]
for col in v_df.columns:
    if col in ["review_id", "Type", "b_rating"]:
        v_df_log[col] = v_df[col]
    else:
        v_df_log[col] = np.log(v_df[col] + 1e-6)  # 对数变换：避免对数0，增加一个很小的常数
v_df_log

/opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,review_id,score,subjectivity,polarity,AO,SE,Type,HV,NoP,words,days,NoF,NoR,volume_b,b_rating
0,h-DEP7jchR5ddJwIcKWfKg,-0.557117,-0.571912,-1.213389,-0.831131,-3.953444,1,9.999995e-07,2.302585e+00,5.468060,8.249314,6.687109e+00,6.759255,7.974877,4.0
1,A3NeldVTkSZhBIpHTG8nLA,-0.608288,-0.385472,-0.902758,-1.139832,-2.060033,1,1.098613e+00,6.931477e-01,4.820282,7.308543,6.603944e+00,5.513429,5.384495,4.0
2,ktdpmCGlzDkQCrCCLylrIg,-0.513654,-0.446286,-0.617420,-1.021648,-1.341955,1,-1.381551e+01,1.386295e+00,4.564348,7.642044,3.951244e+00,6.699500,8.895356,4.5
3,3RmI4QP4sXvNwZzOCzec5g,-0.237556,-0.628607,-1.280930,-0.762138,-8.407343,1,9.999995e-07,1.098613e+00,2.995732,7.171657,9.999995e-07,0.693148,7.259820,4.0
4,G67nazKpUS7ZYNEepeMXAA,-0.428347,-0.532771,-1.307731,-0.884245,-4.883707,1,6.931477e-01,2.079442e+00,5.587249,8.408048,5.455321e+00,4.744932,9.343209,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42934,dG6SJlgd-rBNFWSfmgxUWA,-0.449132,-0.383213,-1.252343,-1.144653,-4.849239,1,9.999995e-07,9.999995e-07,4.442651,7.968666,6.664409e+00,4.867534,7.455877,4.0
42935,mOUegi6l8LoVZHDwpXB5YA,-0.225848,-0.323884,-0.385661,-1.284938,-0.911301,1,-1.381551e+01,9.999995e-07,3.663562,7.999679,9.999995e-07,4.127134,8.036573,4.0
42936,uCkQoCOLEgD8LiidaRN_yg,-0.824260,-0.467823,-1.039363,-0.984470,-2.581242,1,-1.381551e+01,9.999995e-07,5.087596,7.894691,4.653960e+00,4.290459,7.701200,4.5
42937,cgDvrvyB69_tYyW2c3TxaQ,-0.623900,-0.606428,-0.854857,-0.788102,-1.914984,1,9.999995e-07,9.999995e-07,4.709530,7.117206,1.386295e+00,3.401197,8.345218,4.5


In [58]:
# !pip install scikit-learn
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = v_df_log['review_id']
df_scaled = pd.concat([df_scaled, pd.DataFrame(scaler.fit_transform(v_df_log[v_df_log.columns[1:]]), columns=v_df_log.columns[1:])], axis=1)

df_scaled

,review_id,score,subjectivity,polarity,AO,SE,Type,HV,NoP,words,days,NoF,NoR,volume_b,b_rating
0,h-DEP7jchR5ddJwIcKWfKg,-0.713038,0.019902,0.239658,0.141973,-1.266200,0.39876,0.769878,2.085615,0.864341,1.217333,0.658358,1.383376,0.300352,-0.687659
1,A3NeldVTkSZhBIpHTG8nLA,-1.016765,0.391852,0.573845,-0.308515,0.375122,0.39876,0.920620,-0.150232,0.084942,-0.959286,0.643476,0.629437,-2.009299,-0.687659
2,ktdpmCGlzDkQCrCCLylrIg,-0.455066,0.270529,0.880822,-0.136050,0.997596,0.39876,-1.125769,0.812695,-0.222994,-0.187678,0.168780,1.347214,1.121075,1.225604
3,3RmI4QP4sXvNwZzOCzec5g,1.183719,-0.093206,0.166995,0.242654,-5.127106,0.39876,0.769878,0.413044,-2.110333,-1.275993,-0.538288,-2.287663,-0.337211,-0.687659
4,G67nazKpUS7ZYNEepeMXAA,0.051277,0.097989,0.138162,0.064462,-2.072608,0.39876,0.864986,1.775622,1.007748,1.584589,0.437932,0.164364,1.520393,-0.687659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42934,dG6SJlgd-rBNFWSfmgxUWA,-0.072091,0.396360,0.197750,-0.315551,-2.042729,0.39876,0.769878,-1.113159,-0.369418,0.568011,0.654296,0.238559,-0.162401,-0.687659
42935,mOUegi6l8LoVZHDwpXB5YA,1.253214,0.514723,1.130156,-0.520269,1.370912,0.39876,-1.125769,-1.113159,-1.306809,0.639764,-0.538288,-0.209510,0.355363,-0.687659
42936,uCkQoCOLEgD8LiidaRN_yg,-2.298669,0.227562,0.426881,-0.081795,-0.076692,0.39876,-1.125769,-1.113159,0.406572,0.396858,0.294530,-0.110670,0.056335,1.225604
42937,cgDvrvyB69_tYyW2c3TxaQ,-1.109432,-0.048959,0.625379,0.204764,0.500859,0.39876,0.769878,-1.113159,-0.048312,-1.401974,-0.290213,-0.648827,0.630558,1.225604


In [66]:
df_scaled.rename(columns={"score": "AQ"}, inplace=True)

## models construction
$HV_i = β_0 + β_1AQ_i + β_2AO_i + β_3SE_i + β_4AQ_i ⋅ AO_i + β_5AQ_i ⋅ SE_i + ε + a_b + γ ⋅ Contorl$

In [59]:
# !pip install statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm

In [68]:
model = mixedlm("HV ~ AQ + AO + SE + AQ * AO + AQ * SE + NoP + words + days + NoF + NoR + volume_b + b_rating", 
                df_scaled,
                groups=df_scaled["review_id"],
                re_formula=None)  # 不加入任何随机效应

In [76]:
result = model.fit()
result.summary()

/opt/anaconda3/envs/OnlineReview/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: HV         
No. Observations: 42939   Method:             REML       
No. Groups:       42939   Scale:              0.3853     
Min. group size:  1       Log-Likelihood:     -55383.1745
Max. group size:  1       Converged:          Yes        
Mean group size:  1.0                                    
---------------------------------------------------------
              Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept      0.000    0.004   0.003 0.998 -0.008  0.008
AQ             0.007    0.004   1.597 0.110 -0.002  0.016
AO            -0.012    0.004  -2.832 0.005 -0.021 -0.004
SE             0.020    0.004   4.483 0.000  0.011  0.028
AQ:AO          0.004    0.004   1.169 0.243 -0.003  0.012
AQ:SE         -0.001    0.004  -0.166 0.868 -0.009  0.008
NoP            0.026    0.002  10.994 0.000  0.021  0.030
words          0.252    0.004  61.852 0.000  0.244  0.260
days          -0.080    0.005 -16.328 0.000 -0.089 -0.070
NoF            0.092    0.005  18.441 0.000  0.082  0.102
NoR            0.252    0.005  47.478 0.000  0.242  0.262
volume_b      -0.077    0.005 -16.804 0.000 -0.086 -0.068
b_rating       0.014    0.004   3.066 0.002  0.005  0.022
Group Var      0.385                                     
=========================================================

"""